<a href="https://colab.research.google.com/github/tosuzu1/ee496-malariaDetectionML/blob/master/EE496_Malaria_Deteection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the KERAS CNN implementation for the MALARIA CELL IMAGES DATASET

Breakdown of this notebook:

Loading the dataset: Load the data and import the libraries.

In [0]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import os
import cv2
from PIL import Image
# For CNN model creation
import keras
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras.regularizers import l1, l2, l1_l2
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
SIZE = 64

Using TensorFlow backend.


In [0]:
#This section is to import files from kaggle into collab
from google.colab import files

!pip install -q kaggle

# Upload kaggle API key file
uploaded = files.upload()



In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
# Downlaod data for the nyc_taxi_trip_duration challenge
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria

In [0]:
with ZipFile('cell-images-for-detecting-malaria.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()
!ls

cell_images  cell-images-for-detecting-malaria.zip  kaggle.json  sample_data


In [0]:
infected = os.listdir('./cell_images/cell_images/Parasitized/') 
uninfected = os.listdir('./cell_images/cell_images/Uninfected/')

In [0]:
data = []
labels = []

for i in infected:
    try:
    
        image = cv2.imread("./cell_images/cell_images/Parasitized/"+i)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((64 , 64))
        data.append(np.array(resize_img))
        label = to_categorical(1, num_classes=2)
        labels.append(label)
        
    except AttributeError:
        print('')
    
for u in uninfected:
    try:
        
        image = cv2.imread("./cell_images/cell_images/Uninfected/"+u)
        image_array = Image.fromarray(image , 'RGB')
        resize_img = image_array.resize((64 , 64))
        data.append(np.array(resize_img))
        label = to_categorical(0, num_classes=2)
        labels.append(label)
        
    except AttributeError:
        print('')

labels = np.array(labels)

In [0]:
print(labels)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [0]:
regulization_list = [0, 0.0000000001, 0.000000001, 0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
# probability axis values 
prob = []
# loss axis values 
loss = []
for i in regulization_list:
  classifier = Sequential()
  classifier.add(Convolution2D(32, (3, 3), kernal_regularizer=l1(i), bias_regularizer=l1(i), input_shape = (SIZE, SIZE, 3), activation = 'relu'))
  classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
  classifier.add(BatchNormalization(axis = -1))
  classifier.add(Dropout(0.2))
  classifier.add(Convolution2D(32, (3, 3), kernal_regularizer=l1(i), bias_regularizer=l1(i), activation = 'relu'))
  classifier.add(MaxPooling2D(pool_size = (2, 2), data_format="channels_last"))
  classifier.add(BatchNormalization(axis = -1))
  classifier.add(Dropout(0.2))
  classifier.add(Flatten())
  classifier.add(Dense(kernal_regularizer=l1(i), bias_regularizer=l1(i), activation = 'relu', units=512))
  classifier.add(BatchNormalization(axis = -1))
  classifier.add(Dropout(0.2))
  classifier.add(Dense(kernal_regularizer=l2(i), bias_regularizer=l2(i), activation = 'relu', units=256))
  classifier.add(BatchNormalization(axis = -1))
  classifier.add(Dropout(0.2))
  classifier.add(Dense(kernal_regularizer=l1_l2(i), bias_regularizer=l1_l2(i), activation = 'sigmoid', units=2))
  classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
  print(classifier.summary())

In [0]:
  x_train, x_test = train_test_split(data, test_size=0.2, random_state=1,shuffle = True)
  y_train, y_test = train_test_split(labels, test_size=0.2, random_state=1)
  print(np.array(x_train).shape)

(22046, 64, 64, 3)


In [0]:
  history = classifier.fit(np.array(x_train), 
                         y_train, 
                         batch_size = 64, 
                         verbose = 2, 
                         epochs = 30, 
                         validation_split = 0.1,
                         shuffle = False)
                         
  print("Test_Accuracy: {:.2f}%".format(classifier.evaluate(np.array(x_test), np.array(y_test))[1]*100))
  loss.append(classifier.evaluate(np.array(x_test), np.array(y_test))[1])

Train on 19841 samples, validate on 2205 samples
Epoch 1/50
 - 113s - loss: 0.0117 - acc: 0.9952 - val_loss: 0.2172 - val_acc: 0.9596
Epoch 2/50


In [0]:
fig, (ax1, ax2) = plt.subplots(1, 2)
  
ax1.plot(loss, prob)    
ax1.xlabel('Regret') 
ax1.ylabel('Probability')    
ax1.title('Probability vs. Regret')   
ax1.show()

ax2.plot(regulization_list, loss)    
ax2.xlabel('Lamda') 
ax2.ylabel('Regret')    
ax2.title('Regret vs. Lamda')   
ax2.show()